In [3]:
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import os
import pytz
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException

In [4]:
class Boat_Scrapper():
    def __init__(self):
        self.url = 'https://www.boat-lifestyle.com/'
        self.driver = self.init_driver(False)
    
    ## seting-up chrome webdriver
    def init_driver(self,headless, proxy=None):
        """Initialize an instance of Chrome Driver"""
        chrome_options = webdriver.ChromeOptions()
        if headless is True:
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--headless')
        if proxy is not None:
            chrome_options.add_argument(f'--proxy-server={proxy}')

        chrome_options.add_argument("--start-maximized")
        chrome_options.arguments.extend(["--no-sandbox", "--disable-setuid-sandbox", "--disable-dev-shm-usage", "javascript.enabled"])
   
        driver = webdriver.Chrome(options=chrome_options)
        #driver.set_page_load_timeout(50)
        driver.get(self.url)
        return driver
    
    
    def get_details(self, review):
        print('in get details block')
        time.sleep(10)
#         try:
#             category_name = self.driver.find_element(By.XPATH, value = ".//span[@class='jdgm-rev__author']").text
#             print('Category name is:', category_name)
#         except Exception as e:
#             print("Category name error 'cuz of : ", e)    
#         try:
#             product_name = self.driver.find_elements(By.XPATH, value=".//div[@class='product-heading-container']")[1].text
#             print('Product name is:', product_name)
#         except Exception as e:
#             print("Product name error 'cuz of : ", e) 
#             raise e
        try:
            customer_name = review.find_element(By.XPATH, value = ".//span[@class='jdgm-rev__author']").text
            print('Customer name is:', customer_name)
        except Exception as e:
            print("Customer name error 'cuz of : ", e)
        try:
            review_rating = review.find_element(By.XPATH, './/span[@class="jdgm-rev__rating"]').get_attribute('aria-label')[0]
            print('Review rating is:',review_rating)
        except:
            review_rating = 0
        try:
            review_date = review.find_element(By.XPATH, value=".//span[@class='jdgm-rev__timestamp']").text
            print('Review date is:', review_date)
        except:
            review_date = "Not Found"    
        try:
            review_text = review.find_element(By.XPATH, value = ".//div[@class='jdgm-rev__body']").text.replace("\n", " ")
            print('Review text is:', review_text)
        except:
            review_text = ""


        return([customer_name,review_rating, review_date, review_text])
    
#         columns = ["customer_name", "review_rating", "review_date", "review_text"]
#         data = []
#         df = pd.DataFrame(data, columns=columns)
        
#         # Process each review and add data to the DataFrame
#         for review in reviews_list:
#             details = get_details(review)
#             df = df.append(pd.Series(details, index=columns), ignore_index=True)
            
#         # Write the DataFrame to a CSV file
#         csv_file_path = "Boathead.csv"
#         df.to_csv(csv_file_path, index=False)

#         print("Data written to the CSV file successfully.")
    
    
            
    def find_category(self, driver):
        try:
            categ_elements = driver.find_elements(By.XPATH, value="//*[@id='desktop-menu-1']//li//a[@class='nav-dropdown__link link--faded test']")
            categ_list = [element.get_attribute('href') for element in categ_elements]
            if categ_list:
                print("Categories found:")
                for categ_link in categ_list:
                    print(categ_link)
                return categ_list
            else:
                print("No categories found.")
                return None
        except Exception as e:
            print("Error finding categories:", e)
            return None



    def find_element_retry(self, by, value, max_retries=5):
        """Find element with retries in case of StaleElementReferenceException"""
        retries = 0
        while retries < max_retries:
            try:
                element = self.driver.find_element(by, value)
                return element
            except StaleElementReferenceException:
                retries += 1
                time.sleep(1)  # Add a small delay before retrying
        raise NoSuchElementException(f"Element not found after {max_retries} retries")

    def item_category(self, link):
        try:
            item_link = []
            self.driver.get(link)
            category_name = self.find_element_retry(By.XPATH, "//*//h1[@class='heading h1']//split-lines").text
            print(category_name)
            time.sleep(10)
            item_list = self.driver.find_elements(By.XPATH, "//*//product-item[@class='product-item two-point-o  item-horizontal ']//div[@class='position-relative']//a")
            print(item_list)
#             if item_list:
#                 item_link = [item.get_attribute("href") for item in item_list]  # Return a list of item URLs
            return item_list

        except Exception as e:
            print("Error occurred: ", e)
        
        
    def main_driver(self):
        driver = self.init_driver(False)
        categ_list = self.find_category(driver)
        driver.quit()  # We quit the driver as we don't need it for the next steps
       
        item_list = self.item_category(categ_list[5])

        #print('item list is', item_list)
#                 if item_list:
#                     for item in item_list:
#                         #print(item)
        
        if item_list:
                    for item in item_list:
                        #print(item)
                        time.sleep(10)
                        item_url = item.get_attribute("href")
                        print(item_url)
                        data= []
                        self.driver.get(item_url)
                        while True:
                            time.sleep(5)
                            reviews = self.driver.find_elements(By.XPATH, value = ".//div[@class='jdgm-rev jdgm-divider-top jdgm--done-setup']")
                            print(len(reviews))
                        #         except:
                        #             print('No Review available')
                        #             break

                            for review in reviews:
                                details = self.get_details(review)
                                print(details)
                                data.append(details)

                            try:
                                next_button = self.driver.find_element(By.XPATH, value = ".//a[@class='jdgm-paginate__page jdgm-paginate__next-page']")
                                next_button.click()
                                time.sleep(10)
                            except NoSuchElementException:
                                next_button = None
                                print("Reviews Completed.")
                                break


                        df = pd.DataFrame(data, columns = ['customer_name', 'review_rating', 'review_date', 'review_text'])
                        print(df)
                        try:
                            df.to_csv('BoatHead_Categories.csv', index=False)
                            print("CSV file generated successfully.")
                        except Exception as e:
                            print("An error occurred while generating the CSV file:")
                            print(str(e))

        driver.quit()    
        
        
if __name__ == "__main__":

    start_time = time.time()

    class_caller = Boat_Scrapper()
    print("start time", start_time)
    categories = class_caller.main_driver()

    for category_name, item_list in categories:
        print(category_name)
        for item_url in item_list:
            review_details = class_caller.get_details(item_url)
            if review_details:
                print(review_details)

    print("total time", time.time()-start_time)

start time 1691358335.1252182
Categories found:
https://www.boat-lifestyle.com/collections/true-wireless-earbuds
https://www.boat-lifestyle.com/collections/wireless-earphones
https://www.boat-lifestyle.com/collections/smart-watches
https://www.boat-lifestyle.com/collections/wireless-headphones
https://www.boat-lifestyle.com/collections/wireless-speakers
https://www.boat-lifestyle.com/collections/wired-headphones-1
https://www.boat-lifestyle.com/collections/wired-headphones
https://www.boat-lifestyle.com/collections/home-audio
https://www.boat-lifestyle.com/collections/immortal-gaming
https://www.boat-lifestyle.com/collections/party-pal
https://www.boat-lifestyle.com/collections/misfit-by-boat
https://www.boat-lifestyle.com/collections/chargers
https://www.boat-lifestyle.com/collections/power-banks
https://www.boat-lifestyle.com/collections/cables
https://www.boat-lifestyle.com/collections/car-accessories
https://www.boat-lifestyle.com/collections/boat-super-sale
https://www.boat-lifest

Customer name is: Deepak P
Review rating is: 5
Review date is: 19/03/2022
Review text is: Awesome product
['Deepak P', '5', '19/03/2022', 'Awesome product']
in get details block
Customer name is: Nikunj Raj
Review rating is: 5
Review date is: 18/03/2022
Review text is: Its quality is best.
['Nikunj Raj', '5', '18/03/2022', 'Its quality is best.']
in get details block
Customer name is: Sanya Madaan
Review rating is: 5
Review date is: 17/03/2022
Review text is: the price was great honestly and it works so well except very rarely it shuts down for a second or two but other than that i have no complaints now i do wish i bought the wireless ones but this is amazing lol oh and the delivery was too good i got all the updates via message and it was super quick 10/10 will definitely purchase again!!
['Sanya Madaan', '5', '17/03/2022', 'the price was great honestly and it works so well except very rarely it shuts down for a second or two but other than that i have no complaints now i do wish i b

Customer name is: Devika Thapa
Review rating is: 5
Review date is: 13/07/2021
Review text is: Sounds beautiful. Quality is very good and was worth going for it. No regrets.
['Devika Thapa', '5', '13/07/2021', 'Sounds beautiful. Quality is very good and was worth going for it. No regrets.']
in get details block
Customer name is: Subhajit Chakraborty
Review rating is: 5
Review date is: 12/07/2021
Review text is: Bass is awesome, sound quality is good.
['Subhajit Chakraborty', '5', '12/07/2021', 'Bass is awesome, sound quality is good.']
in get details block
Customer name is: Tarachand Kumawat
Review rating is: 5
Review date is: 12/07/2021
Review text is: Sound quality is very good and microphone performance is also nice. Value for money.
['Tarachand Kumawat', '5', '12/07/2021', 'Sound quality is very good and microphone performance is also nice. Value for money.']
6
in get details block
Customer name is: Nilay Kr
Review rating is: 5
Review date is: 09/07/2021
Review text is: Fast deliver

Customer name is: Jay Patel
Review rating is: 5
Review date is: 15/05/2021
Review text is: Best in sound quality
['Jay Patel', '5', '15/05/2021', 'Best in sound quality']
6
in get details block
Customer name is: Sushil kumar Sharma
Review rating is: 4
Review date is: 14/05/2021
Review text is: Aisa basul
['Sushil kumar Sharma', '4', '14/05/2021', 'Aisa basul']
in get details block
Customer name is: Ramkumar Jayaraman
Review rating is: 5
Review date is: 31/03/2021
Review text is: Excellent
['Ramkumar Jayaraman', '5', '31/03/2021', 'Excellent']
in get details block
Customer name is: Tanvir Shaikh
Review rating is: 5
Review date is: 22/03/2021
Review text is: full volume on win10 & people seating in the next room can hear everything but the bass is good compared to others, build quality is okay. overall great.
['Tanvir Shaikh', '5', '22/03/2021', 'full volume on win10 & people seating in the next room can hear everything but the bass is good compared to others, build quality is okay. over

Customer name is: Niharika Kummithi
Review rating is: 5
Review date is: 27/12/2020
Review text is: BassHeads 900
['Niharika Kummithi', '5', '27/12/2020', 'BassHeads 900']
in get details block
Customer name is: siddu
Review rating is: 5
Review date is: 24/12/2020
Review text is: i use this for my online class,, better sound quality for song its is speech less
['siddu', '5', '24/12/2020', 'i use this for my online class,, better sound quality for song its is speech less']
in get details block
Customer name is: Manoj Singh
Review rating is: 5
Review date is: 11/12/2020
Review text is: Working good as expected while purchasing.
['Manoj Singh', '5', '11/12/2020', 'Working good as expected while purchasing.']
in get details block
Customer name is: Aditya Thakur
Review rating is: 4
Review date is: 09/12/2020
Review text is: Good. This is great headphone in this price range.
['Aditya Thakur', '4', '09/12/2020', 'Good. This is great headphone in this price range.']
in get details block
Customer

Customer name is: Kiran Sathya
Review rating is: 5
Review date is: 17/11/2020
Review text is: good product, on-time delivery
['Kiran Sathya', '5', '17/11/2020', 'good product, on-time delivery']
6
in get details block
Customer name is: jyoti ajbani
Review rating is: 4
Review date is: 16/11/2020
Review text is: BassHeads 900
['jyoti ajbani', '4', '16/11/2020', 'BassHeads 900']
in get details block
Customer name is: Nitesh Kumar
Review rating is: 5
Review date is: 16/11/2020
Review text is: BassHeads 900
['Nitesh Kumar', '5', '16/11/2020', 'BassHeads 900']
in get details block
Customer name is: Megha Rangdal
Review rating is: 5
Review date is: 16/11/2020
Review text is: superb
['Megha Rangdal', '5', '16/11/2020', 'superb']
in get details block
Customer name is: Karthikeyan J
Review rating is: 4
Review date is: 14/11/2020
Review text is: good product for price range.. compact as well
['Karthikeyan J', '4', '14/11/2020', 'good product for price range.. compact as well']
in get details bloc

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=115.0.5790.170)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F4A813+48355]
	(No symbol) [0x00EDC4B1]
	(No symbol) [0x00DE5358]
	(No symbol) [0x00DEE5A3]
	(No symbol) [0x00DE8A78]
	(No symbol) [0x00DE8A56]
	(No symbol) [0x00DE7C03]
	(No symbol) [0x00DEA19B]
	(No symbol) [0x00E3D279]
	(No symbol) [0x00E2A73C]
	(No symbol) [0x00E3C922]
	(No symbol) [0x00E2A536]
	(No symbol) [0x00E082DC]
	(No symbol) [0x00E093DD]
	GetHandleVerifier [0x011AAABD+2539405]
	GetHandleVerifier [0x011EA78F+2800735]
	GetHandleVerifier [0x011E456C+2775612]
	GetHandleVerifier [0x00FD51E0+616112]
	(No symbol) [0x00EE5F8C]
	(No symbol) [0x00EE2328]
	(No symbol) [0x00EE240B]
	(No symbol) [0x00ED4FF7]
	BaseThreadInitThunk [0x765A00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77EA7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77EA7AEE+238]


In [ ]:
############################# Trial 2 (limiting the no. of reviews)##################################

In [26]:
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import random
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [39]:
class Boat_Scrapper():
    def __init__(self):
        self.url = 'https://www.boat-lifestyle.com/'
        self.driver = self.init_driver(False)
    
    ## seting-up chrome webdriver
    def init_driver(self,headless, proxy=None):
        """Initialize an instance of Chrome Driver"""
        chrome_options = webdriver.ChromeOptions()
        if headless is True:
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--headless')
        if proxy is not None:
            chrome_options.add_argument(f'--proxy-server={proxy}')

        chrome_options.add_argument("--start-maximized")
        chrome_options.arguments.extend(["--no-sandbox", "--disable-setuid-sandbox", "--disable-dev-shm-usage", "javascript.enabled"])
   
        driver = webdriver.Chrome(options=chrome_options)
        #driver.set_page_load_timeout(50)
        driver.get(self.url)
        return driver
    
    
    def get_details(self, review):
        print('in get details block')
        time.sleep(10)
        try:
            customer_name = review.find_element(By.XPATH, value = ".//span[@class='jdgm-rev__author']").text
            print('Customer name is:', customer_name)
        except Exception as e:
            print("Customer name error 'cuz of : ", e)
        try:
            review_rating = review.find_element(By.XPATH, './/span[@class="jdgm-rev__rating"]').get_attribute('aria-label')[0]
            print('Review rating is:',review_rating)
        except:
            review_rating = 0
        try:
            review_date = review.find_element(By.XPATH, value=".//span[@class='jdgm-rev__timestamp']").text
            print('Review date is:', review_date)
        except:
            review_date = "Not Found"    
        try:
            review_text = review.find_element(By.XPATH, value = ".//div[@class='jdgm-rev__body']").text.replace("\n", " ")
            print('Review text is:', review_text)
        except:
            review_text = ""

#         category_name = self.driver.find_element(By.XPATH, value="//h1[@class='heading h1']//split-lines").text
        return([customer_name, review_rating, review_date, review_text])
    
    
            
    def find_category(self, driver):
        try:
            categ_elements = driver.find_elements(By.XPATH, value="//*[@id='desktop-menu-1']//li//a[@class='nav-dropdown__link link--faded test']")
            categ_list = [element.get_attribute('href') for element in categ_elements]
            if categ_list:
                print("Categories found:")
                for categ_link in categ_list:
                    print(categ_link)
                return categ_list
            else:
                print("No categories found.")
                return None
        except Exception as e:
            print("Error finding categories:", e)
            return None
        
    def find_element_retry(self, by, value, max_retries=5):
        """Find element with retries in case of StaleElementReferenceException"""
        retries = 0
        while retries < max_retries:
            try:
                element = self.driver.find_element(by, value)
                return element
            except StaleElementReferenceException:
                retries += 1
                time.sleep(1)  # Add a small delay before retrying
        raise NoSuchElementException(f"Element not found after {max_retries} retries")

    def item_category(self, link):
        try:
            item_link = []
            self.driver.get(link)
            self.category_name = self.find_element_retry(By.XPATH, "//*//h1[@class='heading h1']//split-lines").text
            print(self.category_name)
            time.sleep(10)
            item_list = self.driver.find_elements(By.XPATH, "//*//product-item[@class='product-item two-point-o  item-horizontal ']//div[@class='position-relative']//a")
            print("Item list is:",item_list)
            if item_list:
                item_links = [item.get_attribute("href") for item in item_list]  # Return a list of item URLs
                return item_links
        except Exception as e:
            print("Error occurred: ", e)

    def main_driver(self):
        driver = self.init_driver(False)
        categ_list = self.find_category(driver)
        print(categ_list)
        driver.quit()  # We quit the driver as we don't need it for the next steps
        item_links = self.item_category(categ_list[5])
        print(item_links)
#         item_name = self.driver.find_elements(By.XPATH, value=".//div[@class='jdgm-rev__body']").text
        df_list = []
        for item_link in item_links:
            data = []
            self.driver.get(item_link)
            
            # Fetch reviews from the first 2 pages
            count = 0
            while count <= 2:
                reviews = self.driver.find_elements(By.XPATH, value=".//div[@class='jdgm-rev jdgm-divider-top jdgm--done-setup']")
                print("Number of reviews on page", ":", len(reviews))

                for review in reviews:
                    details = self.get_details(review)
                    print("Review details:", details)
                    data.append(details)

                try:
                    next_button = self.driver.find_element(By.XPATH, value=".//a[@class='jdgm-paginate__page jdgm-paginate__next-page']")
                    next_button.click()
                    count += 1
                    time.sleep(10)
                except NoSuchElementException:
                    next_button = None
                    print("Reviews Completed.")
                    break

            # Process data from the first 2 pages for this item_url
            print("Data for", item_link, ":", data)

            df = pd.DataFrame(data, columns = ['customer_name', 'review_rating', 'review_date', 'review_text'])
            df['category_name'] = self.category_name
            df = df[['category_name', 'customer_name', 'review_rating', 'review_date', 'review_text']]
            df_list.append(df)
            
            final_df = pd.concat(df_list)
            final_df.to_csv('BoatHead_Headphones_Reviews.csv', index=False)
            print("CSV file generated successfully.")
        
#         item_url = item_links[0]
#         print(item_url)
#         data= []
#         self.driver.get(item_url)

#                 # Fetch reviews from the first 2 pages
# #                 for page_number in range(2):
# #                     self.driver.get(item_url)
# #                     time.sleep(5)
#         count =0
#         while count<=2:
#             reviews = self.driver.find_elements(By.XPATH, value=".//div[@class='jdgm-rev jdgm-divider-top jdgm--done-setup']")
#             print("Number of reviews on page", ":", len(reviews))

#             for review in reviews:
#                 details = self.get_details(review)
#                 print("Review details:", details)
#                 data.append(details)

#             try:
#                 next_button = self.driver.find_element(By.XPATH, value=".//a[@class='jdgm-paginate__page jdgm-paginate__next-page']")
#                 next_button.click()
#                 count+=1
#                 time.sleep(10)
#             except NoSuchElementException:
#                 next_button = None
#                 print("Reviews Completed.")
#                 break

#         # Process data from the first 2 pages for this item_url
#         print("Data for", item_url, ":", data)          


#         df = pd.DataFrame(data, columns = ['customer_name', 'review_rating', 'review_date', 'review_text'])
#         print(df)
#         try:
#             df.to_csv('BoatHead_Categories.csv', index=False)
#             print("CSV file generated successfully.")
#         except Exception as e:
#             print("An error occurred while generating the CSV file:")
#             print(str(e))

        driver.quit()
        
        
if __name__ == "__main__":
    start_time = time.time()
    class_caller = Boat_Scrapper()
    print("start time", start_time)
    categories = class_caller.main_driver()
    
    if categories:
        for category_name, item_links in categories:
            print("Category name:",category_name)
            for item_url in item_links:
                review_details = class_caller.get_details(item_url)
                if review_details:
                    print(review_details)
    else:
        print("Categories not found")

    print("total time", time.time()-start_time)

start time 1691409160.3814156
Categories found:
https://www.boat-lifestyle.com/collections/true-wireless-earbuds
https://www.boat-lifestyle.com/collections/wireless-earphones
https://www.boat-lifestyle.com/collections/smart-watches
https://www.boat-lifestyle.com/collections/wireless-headphones
https://www.boat-lifestyle.com/collections/wireless-speakers
https://www.boat-lifestyle.com/collections/wired-headphones-1
https://www.boat-lifestyle.com/collections/wired-headphones
https://www.boat-lifestyle.com/collections/home-audio
https://www.boat-lifestyle.com/collections/immortal-gaming
https://www.boat-lifestyle.com/collections/party-pal
https://www.boat-lifestyle.com/collections/misfit-by-boat
https://www.boat-lifestyle.com/collections/chargers
https://www.boat-lifestyle.com/collections/power-banks
https://www.boat-lifestyle.com/collections/cables
https://www.boat-lifestyle.com/collections/car-accessories
https://www.boat-lifestyle.com/collections/boat-super-sale
https://www.boat-lifest

Number of reviews on page : 6
in get details block
Customer name is: Bageeswaran .
Review rating is: 5
Review date is: 07/10/2022
Review text is: Both sound and product quality is great
Review details: ['Bageeswaran .', '5', '07/10/2022', 'Both sound and product quality is great']
in get details block
Customer name is: Vishnu prasad
Review rating is: 4
Review date is: 28/09/2022
Review text is: good
Review details: ['Vishnu prasad', '4', '28/09/2022', 'good']
in get details block
Customer name is: Sankalp Mathur
Review rating is: 5
Review date is: 09/06/2022
Review text is: One of the best headphones I've ever used. Absolutely love them. I use them almost 15-16 hours a day not just for gaming but for all the other kinds of stuff like online classes and youtube videos. I would surely recommend buying this product.
Review details: ['Sankalp Mathur', '5', '09/06/2022', "One of the best headphones I've ever used. Absolutely love them. I use them almost 15-16 hours a day not just for gaming

In [ ]:
########################## For all categories ############################

In [41]:
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import random
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [43]:
class Boat_Scrapper():
    def __init__(self):
        self.url = 'https://www.boat-lifestyle.com/'
        self.driver = self.init_driver(False)
    
    ## seting-up chrome webdriver
    def init_driver(self,headless, proxy=None):
        """Initialize an instance of Chrome Driver"""
        chrome_options = webdriver.ChromeOptions()
        if headless is True:
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--headless')
        if proxy is not None:
            chrome_options.add_argument(f'--proxy-server={proxy}')

        chrome_options.add_argument("--start-maximized")
        chrome_options.arguments.extend(["--no-sandbox", "--disable-setuid-sandbox", "--disable-dev-shm-usage", "javascript.enabled"])
   
        driver = webdriver.Chrome(options=chrome_options)
        #driver.set_page_load_timeout(50)
        driver.get(self.url)
        return driver
    
    
    def get_details(self, review):
        print('in get details block')
        time.sleep(10)
        try:
            customer_name = review.find_element(By.XPATH, value = ".//span[@class='jdgm-rev__author']").text
            print('Customer name is:', customer_name)
        except Exception as e:
            print("Customer name error 'cuz of : ", e)
        try:
            review_rating = review.find_element(By.XPATH, './/span[@class="jdgm-rev__rating"]').get_attribute('aria-label')[0]
            print('Review rating is:',review_rating)
        except:
            review_rating = 0
        try:
            review_date = review.find_element(By.XPATH, value=".//span[@class='jdgm-rev__timestamp']").text
            print('Review date is:', review_date)
        except:
            review_date = "Not Found"    
        try:
            review_text = review.find_element(By.XPATH, value = ".//div[@class='jdgm-rev__body']").text.replace("\n", " ")
            print('Review text is:', review_text)
        except:
            review_text = ""

        return([customer_name, review_rating, review_date, review_text])
    
    
            
    def find_category(self, driver):
        try:
            categ_elements = driver.find_elements(By.XPATH, value="//*[@id='desktop-menu-1']//li//a[@class='nav-dropdown__link link--faded test']")
            categ_list = [element.get_attribute('href') for element in categ_elements]
            if categ_list:
                print("Categories found:")
                for categ_link in categ_list:
                    print(categ_link)
                return categ_list
            else:
                print("No categories found.")
                return None
        except Exception as e:
            print("Error finding categories:", e)
            return None
        
    def find_element_retry(self, by, value, max_retries=5):
        """Find element with retries in case of StaleElementReferenceException"""
        retries = 0
        while retries < max_retries:
            try:
                element = self.driver.find_element(by, value)
                return element
            except StaleElementReferenceException:
                retries += 1
                time.sleep(1)  # Add a small delay before retrying
        raise NoSuchElementException(f"Element not found after {max_retries} retries")

    def item_category(self, link):
        try:
            item_link = []
            self.driver.get(link)
            self.category_name = self.find_element_retry(By.XPATH, "//*//h1[@class='heading h1']//split-lines").text
            print(self.category_name)
            time.sleep(10)
            item_list = self.driver.find_elements(By.XPATH, "//*//product-item[@class='product-item two-point-o  item-horizontal ']//div[@class='position-relative']//a")
            print("Item list is:",item_list)
            if item_list:
                item_links = [item.get_attribute("href") for item in item_list]  # Return a list of item URLs
                return item_links

        except Exception as e:
            print("Error occurred: ", e)

    def main_driver(self):
        driver = self.init_driver(False)
        categ_list = self.find_category(self.driver)
        driver.quit()  # We quit the driver as we don't need it for the next steps
        for categ in categ_list:
            item_links = self.item_category(categ)
            print(item_links)

            df_list = []
            if item_links:
                for item_link in item_links:
                    data = []
                    self.driver.get(item_link)

                    # Fetch reviews from the first 2 pages
                    count = 0
                    while count <= 1:
                        reviews = self.driver.find_elements(By.XPATH, value=".//div[@class='jdgm-rev jdgm-divider-top jdgm--done-setup']")
                        print("Number of reviews on page", ":", len(reviews))

                        for review in reviews:
                            details = self.get_details(review)
                            print("Review details:", details)
                            data.append(details)

                        try:
                            next_button = self.driver.find_element(By.XPATH, value=".//a[@class='jdgm-paginate__page jdgm-paginate__next-page']")
                            next_button.click()
                            count += 1
                            time.sleep(10)
                        except NoSuchElementException:
                            next_button = None
                            print("Reviews Completed.")
                            break

                    # Process data from the first 2 pages for this item_url
                    print("Data for", item_link, ":", data)

                    df = pd.DataFrame(data, columns = ['customer_name', 'review_rating', 'review_date', 'review_text'])
                    df['category_name'] = self.category_name
                    df = df[['category_name', 'customer_name', 'review_rating', 'review_date', 'review_text']]
                    df_list.append(df)

                    final_df = pd.concat(df_list)
                    final_df.to_csv('BoatHead_Reviews.csv', index=False)
                    print("CSV file generated successfully.")
        

            self.driver.quit()
        
        
if __name__ == "__main__":
    start_time = time.time()
    class_caller = Boat_Scrapper()
    print("start time", start_time)
    categories = class_caller.main_driver()
    
    if categories:
        for category_name, item_links in categories:
            print("Category name:",category_name)
            for item_url in item_links:
                review_details = class_caller.get_details(item_url)
                if review_details:
                    print(review_details)
    else:
        print("Categories not found")

    print("total time", time.time()-start_time)

start time 1691414525.4185183
Categories found:
https://www.boat-lifestyle.com/collections/true-wireless-earbuds
https://www.boat-lifestyle.com/collections/wireless-earphones
https://www.boat-lifestyle.com/collections/smart-watches
https://www.boat-lifestyle.com/collections/wireless-headphones
https://www.boat-lifestyle.com/collections/wireless-speakers
https://www.boat-lifestyle.com/collections/wired-headphones-1
https://www.boat-lifestyle.com/collections/wired-headphones
https://www.boat-lifestyle.com/collections/home-audio
https://www.boat-lifestyle.com/collections/immortal-gaming
https://www.boat-lifestyle.com/collections/party-pal
https://www.boat-lifestyle.com/collections/misfit-by-boat
https://www.boat-lifestyle.com/collections/chargers
https://www.boat-lifestyle.com/collections/power-banks
https://www.boat-lifestyle.com/collections/cables
https://www.boat-lifestyle.com/collections/car-accessories
https://www.boat-lifestyle.com/collections/boat-super-sale
https://www.boat-lifest

Review date is: 27/11/2022
Review text is: Very nice product
Review details: ['Raval shivani', '5', '27/11/2022', 'Very nice product']
Data for https://www.boat-lifestyle.com/products/airdopes-131 : [['Divyanshu Bhardwaj', '5', '30/06/2023', 'boAt Airdopes 131 | Wireless Earbuds with upto 60 Hours Playback, 13mm Drivers, IWP Technology, 650mAh Charging Case'], ['V.S.', '5', '13/12/2022', 'Good product, sound quality is also good'], ['Biswaroop Biswas', '5', '12/12/2022', 'Product is simply awesome. Loved this.'], ['Avdesh Pal', '5', '09/12/2022', 'Worst product no exchange no return policy. Not buy again any product from boat.'], ['Ankit Paramanya', '5', '30/11/2022', 'So cool'], ['Ashish Jain', '5', '29/11/2022', 'good quality'], ['Sumit Yadav', '5', '28/11/2022', 'GOOD'], ['Manish Sharma', '5', '28/11/2022', 'Truly amazing earbuds.'], ['vishwas.dhuri', '5', '28/11/2022', 'used for 4 days .....till date experience was too good.....hope it will carry the same.'], ['Ramjan .', '5', '27/

Customer name is: Abhishek a n
Review rating is: 5
Review date is: 10/11/2022
Review text is: Fantastic product
Review details: ['Abhishek a n', '5', '10/11/2022', 'Fantastic product']
Data for https://www.boat-lifestyle.com/products/airdopes-161 : [['Mintu Kumar', '5', '30/06/2023', 'boAt Airdopes 161 | Wireless Earbuds with Massive Playback of upto 17 Hour, IPX5 Water & Sweat Resistance, IWP Technology, Type C Interface'], ['Tosif Hussain', '5', '03/02/2023', 'Amazing sound quality and battery life'], ['Prince raj', '5', '07/12/2022', 'Nice sound within good range'], ['Abhishek Kumar', '5', '18/11/2022', 'Amazing product .'], ['Jagjeet Singh', '5', '18/11/2022', 'Value for money product'], ['kavya mali', '5', '17/11/2022', 'Best 👍'], ['Tirkala Akshith', '5', '16/11/2022', 'boAt Airdopes 161 | Wireless Earbuds with Massive Playback of upto 17 Hour, IPX5 Water & Sweat Resistance, IWP Technology, Type C Interface'], ['HARSH SHARMA', '5', '15/11/2022', 'Fantastic completely satisfy'], ['

Customer name is: Sahil Shetty
Review rating is: 5
Review date is: 01/05/2023
Review text is: Product is very good and delivery was very fast
Review details: ['Sahil Shetty', '5', '01/05/2023', 'Product is very good and delivery was very fast']
in get details block
Customer name is: Sachit Rathi
Review rating is: 5
Review date is: 28/04/2023
Review text is: great headphones
Review details: ['Sachit Rathi', '5', '28/04/2023', 'great headphones']
in get details block
Customer name is: Rohithh
Review rating is: 5
Review date is: 26/04/2023
Review text is: Satisfied low latency
Review details: ['Rohithh', '5', '26/04/2023', 'Satisfied low latency']
in get details block
Customer name is: Alen Paul
Review rating is: 5
Review date is: 25/04/2023
Review text is: boAt Immortal 121 | Bluetooth Gaming Wireless Earbuds with BEAST™️ Mode (40ms Low Latency), ASAP™️ Charge, 40 Hours Playback, & Blazing RGB Lights
Review details: ['Alen Paul', '5', '25/04/2023', 'boAt Immortal 121 | Bluetooth Gaming W

Customer name is: P...
Review rating is: 5
Review date is: 17/06/2023
Review text is: If you true music or bass lover then its 13mm drivers is really adequate. And for calling experience with 4 mic is superb and 80% noise cancellation with clear call quality. Battery backup is also very excellent. I specially use for bass music with clear sound and for calling that is awesome. I rated totally 9/10 overall experience.
Review details: ['P...', '5', '17/06/2023', 'If you true music or bass lover then its 13mm drivers is really adequate. And for calling experience with 4 mic is superb and 80% noise cancellation with clear call quality. Battery backup is also very excellent. I specially use for bass music with clear sound and for calling that is awesome. I rated totally 9/10 overall experience.']
in get details block
Customer name is: Saurav singh Solanki
Review rating is: 5
Review date is: 15/06/2023
Review text is: Very high level decent base, awesome sound quality and fit is superb..!!
R

...

Number of reviews on page : 6
in get details block
Customer name is: P.G.
Review rating is: 5
Review date is: 12/12/2022
Review text is: Very nice product Plz buy this
Review details: ['P.G.', '5', '12/12/2022', 'Very nice product Plz buy this']
in get details block
Customer name is: Adhitthan T
Review rating is: 5
Review date is: 10/12/2022
Review text is: Awsome battery backup.. Really worh under 1500.. 😍
Review details: ['Adhitthan T', '5', '10/12/2022', 'Awsome battery backup.. Really worh under 1500.. 😍']
in get details block
Customer name is: Sachin Thakur
Review rating is: 1
Review date is: 05/12/2022
Review text is: Good product
Review details: ['Sachin Thakur', '1', '05/12/2022', 'Good product']
in get details block
Customer name is: marimuthu kumaravel
Review rating is: 5
Review date is: 29/11/2022
Review text is: Good looking
Review details: ['marimuthu kumaravel', '5', '29/11/2022', 'Good looking']
in get details block
Customer name is: Gaurav .
Review rating is: 5
Review d

Number of reviews on page : 6
in get details block
Customer name is: Gracy Karn
Review rating is: 5
Review date is: 08/12/2022
Review text is: It's nice. Sound is amazing but there's a softness in the battery showing area. When I opened it, suddenly it's turn out a scratch on it. However, it's good
Review details: ['Gracy Karn', '5', '08/12/2022', "It's nice. Sound is amazing but there's a softness in the battery showing area. When I opened it, suddenly it's turn out a scratch on it. However, it's good"]
in get details block
Customer name is: Akshay dinkar thombare
Review rating is: 1
Review date is: 30/11/2022
Review text is: Replace ment
Review details: ['Akshay dinkar thombare', '1', '30/11/2022', 'Replace ment']
in get details block
Customer name is: Abhishek Abhishek
Review rating is: 5
Review date is: 28/11/2022
Review text is: Fastest delivery
Review details: ['Abhishek Abhishek', '5', '28/11/2022', 'Fastest delivery']
in get details block
Customer name is: Syed Ahmed
Review rat

Customer name is: Aniket maruti devadkar
Review rating is: 5
Review date is: 14/11/2022
Review text is: boAt Airdopes 183 | Wireless Bluetooth Earbuds with 10mm Driver, Upto 90 min Playback in 10 min Charge, ASAP Charge Technology, BEAST™ mode
Review details: ['Aniket maruti devadkar', '5', '14/11/2022', 'boAt Airdopes 183 | Wireless Bluetooth Earbuds with 10mm Driver, Upto 90 min Playback in 10 min Charge, ASAP Charge Technology, BEAST™ mode']
Number of reviews on page : 6
in get details block
Customer name is: Jeet Nahar
Review rating is: 5
Review date is: 12/11/2022
Review text is: Got this at 1200. Really good product. Fitting is the main mention of this product. Sound quality is too good
Review details: ['Jeet Nahar', '5', '12/11/2022', 'Got this at 1200. Really good product. Fitting is the main mention of this product. Sound quality is too good']
in get details block
Customer name is: Kushal Kumar
Review rating is: 5
Review date is: 12/11/2022
Review text is: BEST PRODUCT BASS IS

Review text is: VERY NICE PRODUCT
Review details: ['Abhinav Gogna', '5', '07/12/2022', 'VERY NICE PRODUCT']
in get details block
Customer name is: V.R.
Review rating is: 5
Review date is: 27/11/2022
Review text is: Good
Review details: ['V.R.', '5', '27/11/2022', 'Good']
in get details block
Customer name is: Ritul .
Review rating is: 5
Review date is: 21/11/2022
Review text is: Bass is good and beast mode is awesome...I will suggested to for gaming this is awe some
Review details: ['Ritul .', '5', '21/11/2022', 'Bass is good and beast mode is awesome...I will suggested to for gaming this is awe some']
in get details block
Customer name is: Shalom karmarkar
Review rating is: 5
Review date is: 20/11/2022
Review text is: Excellent
Review details: ['Shalom karmarkar', '5', '20/11/2022', 'Excellent']
in get details block
Customer name is: Aditya prasad
Review rating is: 5
Review date is: 18/11/2022
Review text is: Maximum times i face problems to connect earbuds properly. When i turn on bl

Error occurred:  HTTPConnectionPool(host='localhost', port=53255): Max retries exceeded with url: /session/6205d30b6c82f7f74b3ecbafcc30f4ab/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000285CF264820>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
None
Error occurred:  HTTPConnectionPool(host='localhost', port=53255): Max retries exceeded with url: /session/6205d30b6c82f7f74b3ecbafcc30f4ab/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000285CF25BE50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
None
Error occurred:  HTTPConnectionPool(host='localhost', port=53255): Max retries exceeded with url: /session/6205d30b6c82f7f74b3ecbafcc30f4ab/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000285CF264F70>: F

In [ ]:
################# Limiting the no. of items under each category ###########################

In [44]:
import time
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import random
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
class Boat_Scrapper():
    def __init__(self):
        self.url = 'https://www.boat-lifestyle.com/'
        self.driver = self.init_driver(False)
    
    ## seting-up chrome webdriver
    def init_driver(self,headless, proxy=None):
        """Initialize an instance of Chrome Driver"""
        chrome_options = webdriver.ChromeOptions()
        if headless is True:
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--headless')
        if proxy is not None:
            chrome_options.add_argument(f'--proxy-server={proxy}')

        chrome_options.add_argument("--start-maximized")
        chrome_options.arguments.extend(["--no-sandbox", "--disable-setuid-sandbox", "--disable-dev-shm-usage", "javascript.enabled"])
   
        driver = webdriver.Chrome(options=chrome_options)
        #driver.set_page_load_timeout(50)
        driver.get(self.url)
        return driver
    
    
    def get_details(self, review):
        print('in get details block')
        time.sleep(10)
        try:
            customer_name = review.find_element(By.XPATH, value = ".//span[@class='jdgm-rev__author']").text
            print('Customer name is:', customer_name)
        except Exception as e:
            print("Customer name error 'cuz of : ", e)
        try:
            review_rating = review.find_element(By.XPATH, './/span[@class="jdgm-rev__rating"]').get_attribute('aria-label')[0]
            print('Review rating is:',review_rating)
        except:
            review_rating = 0
        try:
            review_date = review.find_element(By.XPATH, value=".//span[@class='jdgm-rev__timestamp']").text
            print('Review date is:', review_date)
        except:
            review_date = "Not Found"    
        try:
            review_text = review.find_element(By.XPATH, value = ".//div[@class='jdgm-rev__body']").text.replace("\n", " ")
            print('Review text is:', review_text)
        except:
            review_text = ""

        return([customer_name, review_rating, review_date, review_text])
    
    
            
    def find_category(self, driver):
        try:
            categ_elements = driver.find_elements(By.XPATH, value="//*[@id='desktop-menu-1']//li//a[@class='nav-dropdown__link link--faded test']")
            categ_list = [element.get_attribute('href') for element in categ_elements]
            if categ_list:
                print("Categories found:")
                for categ_link in categ_list:
                    print(categ_link)
                return categ_list
            else:
                print("No categories found.")
                return None
        except Exception as e:
            print("Error finding categories:", e)
            return None
        
    def find_element_retry(self, by, value, max_retries=5):
        """Find element with retries in case of StaleElementReferenceException"""
        retries = 0
        while retries < max_retries:
            try:
                element = self.driver.find_element(by, value)
                return element
            except StaleElementReferenceException:
                retries += 1
                time.sleep(1)  # Add a small delay before retrying
        raise NoSuchElementException(f"Element not found after {max_retries} retries")

    def item_category(self, link):
        try:
            item_link = []
            self.driver.get(link)
            self.category_name = self.find_element_retry(By.XPATH, "//*//h1[@class='heading h1']//split-lines").text
            print(self.category_name)
            time.sleep(7)
            item_list = self.driver.find_elements(By.XPATH, "//*//product-item[@class='product-item two-point-o  item-horizontal ']//div[@class='position-relative']//a")
            print("Item list is:",item_list)
            item_names = []  # Create an empty list to store item names
            if item_list:
#                 for item in item_list:
#                     item_name = item.find_element(By.XPATH, ".//a[@class='product-item-meta__title tile-title']").text
#                     item_names.append(item_name)  # Append each item name to the list

#                 print("Item names:", item_names)
                item_links = [item.get_attribute("href") for item in item_list]  # Return a list of item URLs
                item_names = [item.text for item in item_list]
                return item_links, item_names
                

        except Exception as e:
            print("Error occurred: ", e)

    def main_driver(self):
        try:
#             driver = self.init_driver(False)
            categ_list = self.find_category(self.driver)
            print(categ_list)
#             driver.quit()  # We quit the driver as we don't need it for the next steps
            for categ in categ_list:
                try:
#                     driver = self.init_driver(False)
                    item_links, item_names = self.item_category(categ)
                    print(item_links)
                    print(item_names)

                    df_list = []
                    if item_links:
                        for item_link in item_links[:2]:
                            data = []
                            self.driver.get(item_link)

                            # Fetch reviews from the first 2 pages
                            count = 0
                            while count <= 1:
                                reviews = self.driver.find_elements(By.XPATH, value=".//div[@class='jdgm-rev jdgm-divider-top jdgm--done-setup']")
                                print("Number of reviews on page", ":", len(reviews))

                                for review in reviews:
                                    details = self.get_details(review)
                                    print("Review details:", details)
                                    data.append(details)

                                try:
                                    next_button = self.driver.find_element(By.XPATH, value=".//a[@class='jdgm-paginate__page jdgm-paginate__next-page']")
                                    next_button.click()
                                    count += 1
                                    time.sleep(10)
                                except NoSuchElementException:
                                    next_button = None
                                    print("Reviews Completed.")
                                    break

                            # Process data from the first 2 pages for this item_url
                            print("Data for", item_link, ":", data)

                            df = pd.DataFrame(data, columns = ['customer_name', 'review_rating', 'review_date', 'review_text'])
                            df['category_name'] = self.category_name
                            df = df[['category_name', 'customer_name', 'review_rating', 'review_date', 'review_text']]
                            df_list.append(df)

                            final_df = pd.concat(df_list)
                            final_df.to_csv('BoatHead_Reviews_Final.csv', index=False)
                            print("CSV file generated successfully.")


#                     self.driver.quit()
                except Exception as e:
                    print("Error occurred while processing category:", categ, "-", e)
                    
        except Exception as e:
            print("Error occurred in main_driver:", e) 
            
    self.driver.quit()        
        
if __name__ == "__main__":
    start_time = time.time()
    class_caller = Boat_Scrapper()
    print("start time", start_time)
    categories = class_caller.main_driver()
    
    if categories:
        for category_name, item_links in categories:
            print("Category name:",category_name)
            for item_url in item_links:
                review_details = class_caller.get_details(item_url)
                if review_details:
                    print(review_details)
    else:
        print("Categories not found")

    print("total time", time.time()-start_time)